In [1]:
%%bash
pip install --upgrade duckdb
pip install --upgrade jupysql
pip install --upgrade duckdb-engine

In [1]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:////home/marialejandramantillab/dmeyf.db

There's a new jupysql version available (0.10.2), you're running 0.10.1. To upgrade: pip install jupysql --upgrade


In [2]:
%%sql
create or replace table competencia_02 as
select
    *
from read_csv_auto('~/buckets/b1/datasets/competencia_03_crudo.csv.gz')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [3]:
%%sql
create or replace table targets as
with periodos as (
    select distinct foto_mes from competencia_02
), clientes as (
    select distinct numero_de_cliente from competencia_02
), todo as (
    select numero_de_cliente, foto_mes from clientes cross join periodos
), clase_ternaria as (
    select
        t.numero_de_cliente
        , t.foto_mes
        , if(c.numero_de_cliente is null, 0, 1) as mes_0
        , lead(mes_0, 1) over (partition by t.numero_de_cliente order by foto_mes) as mes_1
        , lead(mes_0, 2) over (partition by t.numero_de_cliente order by foto_mes) as mes_2
        , case             
            when mes_0 = 1 and mes_1 is null then 'CONTINUA'
            when mes_0 = 1 and mes_1 = 1 and mes_2 is null then 'CONTINUA'
            when mes_0 = 1 and mes_1 = 1 and mes_2 = 1 then 'CONTINUA'
            when mes_0 = 1 and mes_1 = 1 and mes_2 = 0 then 'BAJA+2'            
            when mes_0 = 1 and mes_1 = 0 then 'BAJA+1'            
            
            else 'OTHER'
        end
        as clase_ternaria
    from todo t    
    left join competencia_02 c using (numero_de_cliente, foto_mes)
) select
  foto_mes
  , numero_de_cliente  
  , clase_ternaria
from clase_ternaria where mes_0 = 1

,Success


In [4]:
%%sql
select numero_de_cliente, foto_mes, clase_ternaria from competencia_02 where numero_de_cliente=35299456;

RuntimeError: If using snippets, you may pass the --with argument explicitly.
For more details please refer: https://jupysql.ploomber.io/en/latest/compose.html#with-argument


Original error message from DB driver:
(duckdb.BinderException) Binder Error: Referenced column "clase_ternaria" not found in FROM clause!
Candidate bindings: "competencia_02.catm_trx"
LINE 1: select numero_de_cliente, foto_mes, clase_ternaria from competencia_02 wher...
                                            ^
[SQL: select numero_de_cliente, foto_mes, clase_ternaria from competencia_02 where numero_de_cliente=35299456;]
(Background on this error at: https://sqlalche.me/e/20/f405)

If you need help solving this issue, send us a message: https://ploomber.io/community


In [5]:
%%sql
select numero_de_cliente from targets where clase_ternaria = 'BAJA+2' limit 100;

,numero_de_cliente
0,29282120
1,30142191
2,30471363
3,30484010
4,30519442
...,...
95,73592574
96,73605289
97,73649969
98,73659901


In [6]:
%%sql
alter table competencia_02 add column clase_ternaria VARCHAR(10)

,Success


In [7]:
%%sql
update competencia_02
set clase_ternaria = targets.clase_ternaria
from targets
where competencia_02.numero_de_cliente = targets.numero_de_cliente and competencia_02.foto_mes = targets.foto_mes;

,Success


In [8]:
%%sql
select distinct foto_mes, clase_ternaria, count(*) from competencia_02 group by foto_mes, clase_ternaria order by foto_mes, clase_ternaria;

,foto_mes,clase_ternaria,count_star()
0,201901,BAJA+1,660
1,201901,BAJA+2,732
2,201901,CONTINUA,123472
3,201902,BAJA+1,735
4,201902,BAJA+2,698
...,...,...,...
91,202107,BAJA+2,1268
92,202107,CONTINUA,162336
93,202108,BAJA+1,1275
94,202108,CONTINUA,163706


In [9]:
%%sql
delete from competencia_02
where  foto_mes != '202109' and not clase_ternaria in ('CONTINUA', 'BAJA+1', 'BAJA+2');

,Success


In [9]:
%%sql
update competencia_02 set clase_ternaria = null where foto_mes = '202109';

,Success


In [10]:
%%sql
copy competencia_02 to '~/buckets/b1/datasets/competencia_03.csv.gz' (FORMAT CSV, HEADER)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success
